In [1]:
import pandas as pd
from scipy.stats import chisquare

In [2]:
def get_years_dfs(misinfotext_path: str, fakespeak_path: str):
    misinfotext_file = pd.ExcelFile(misinfotext_path)
    
    for sheet in misinfotext_file.sheet_names:
        df = pd.read_excel(misinfotext_file, sheet_name=sheet)
        df["year"] = sheet
        yield df
    
    fakespeak_file = pd.ExcelFile(fakespeak_path)
    
    for sheet in fakespeak_file.sheet_names:
        df = pd.read_excel(fakespeak_file, sheet_name=sheet)
        df["year"] = sheet
        yield df

In [3]:
def build_chisquare_table(df: pd.DataFrame, ngrams: list[str]):
    # Fill in 0 if ngram doesn't exist in this df
    counts = df.set_index("ngram_text")["ngram_count"]\
                .reindex(ngrams, fill_value=0)

    table = pd.DataFrame({
        "ngram_text": ngrams,
        "ngram_count": counts.values
    })

    table["year"] = df["year"].iloc[0]

    return table

In [4]:
def run_stats_test(misinfotext_path: str, fakespeak_path: str, min_significant_pvalue=0.05):
    years_dfs = list(get_years_dfs(misinfotext_path, fakespeak_path))

    for df in years_dfs:
        df.sort_values("ngram_text", inplace=True)

    all_ngrams = sorted(set([
        ngram 
        for df in years_dfs
        for ngram in df["ngram_text"] 
    ]))

    tables = [build_chisquare_table(df, all_ngrams) for df in years_dfs]

    chisquare_result = chisquare([df["ngram_count"] for df in tables])

    num_insignificant_ngrams = sum(
        1 for pvalue in chisquare_result.pvalue 
        if pvalue > min_significant_pvalue
    )
    num_significant_ngrams = len(all_ngrams) - num_insignificant_ngrams

    print("There are", num_insignificant_ngrams, "ngrams that did not change significantly, and", 
          num_significant_ngrams, "ngrams that changed significantly")
    print("Total", len(all_ngrams), "ngrams")

    return pd.DataFrame({
        "ngram_text": all_ngrams,
        "statistic": chisquare_result.statistic,
        "pvalue": chisquare_result.pvalue
    })

#### Body text

In [5]:
results_all_types = run_stats_test(    
    misinfotext_path="./data/MisInfoText/Analysis_output/MisInfoText_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_ngrams.xlsx"
)
results_all_types

There are 0 ngrams that did not change significantly, and 1409 ngrams that changed significantly
Total 1409 ngrams


,ngram_text,statistic,pvalue
0,09 brown voted,75.0,5.662025e-10
1,09 brown voted yea,60.0,2.522085e-07
2,09 brown voted yea brown,60.0,2.522085e-07
3,10 brown voted yea,45.0,7.657266e-05
4,10 brown voted yea brown,45.0,7.657266e-05
...,...,...,...
1404,zika virus,180.0,2.377396e-30
1405,zone remind people every,45.0,7.657266e-05
1406,zone remind people every day,45.0,7.657266e-05
1407,𝒃𝒍𝒐𝒐𝒅 𝒑𝒓𝒆𝒔𝒔𝒖𝒓𝒆 𝒘𝒊𝒍𝒍 𝒃𝒆,30.0,1.192150e-02


In [6]:
results_news_and_blog = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/news_and_blog/MisInfoText_news_and_blog_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/news_and_blog/Fakespeak_news_and_blog_ngrams.xlsx"
)
results_news_and_blog

There are 72 ngrams that did not change significantly, and 1413 ngrams that changed significantly
Total 1485 ngrams


,ngram_text,statistic,pvalue
0,10 10 per,60.0,2.522085e-07
1,10 10 per hour,60.0,2.522085e-07
2,10 national guard troops,45.0,7.657266e-05
3,10 per hour,60.0,2.522085e-07
4,100 person years,240.0,1.416373e-42
...,...,...,...
1480,zone remind people every,45.0,7.657266e-05
1481,zone remind people every day,45.0,7.657266e-05
1482,zone thousands coming afghanistan al,15.0,4.514172e-01
1483,zoo radio dj libertarian,15.0,4.514172e-01


In [7]:
results_press_release = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/press_release/MisInfoText_press_release_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/press_release/Fakespeak_press_release_ngrams.xlsx"
)
results_press_release

There are 310 ngrams that did not change significantly, and 1379 ngrams that changed significantly
Total 1689 ngrams


,ngram_text,statistic,pvalue
0,09 brown,90.0,9.918667e-13
1,09 brown voted,75.0,5.662025e-10
2,09 brown voted yea,60.0,2.522085e-07
3,09 brown voted yea brown,60.0,2.522085e-07
4,10 brown voted,45.0,7.657266e-05
...,...,...,...
1684,zones efficient ways vessels reduce,15.0,4.514172e-01
1685,zones federal waters identify protect,15.0,4.514172e-01
1686,zones waterways great lakes coasts,15.0,4.514172e-01
1687,zukowski every republican senate,15.0,4.514172e-01


In [8]:
results_social_media = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/social_media/MisInfoText_social_media_ngrams.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/social_media/Fakespeak_social_media_ngrams.xlsx"
)
results_social_media

There are 412 ngrams that did not change significantly, and 954 ngrams that changed significantly
Total 1366 ngrams


,ngram_text,statistic,pvalue
0,11 year old,52.0,1.347814e-06
1,1233 hhh,39.0,1.999436e-04
2,13 year old,78.0,2.619270e-11
3,13 year old girl,52.0,1.347814e-06
4,15 credit card,39.0,1.999436e-04
...,...,...,...
1361,𝘁𝗼 𝗛𝗮𝗶𝘁𝗶𝗮𝗻 𝗖𝗮𝗻𝗻𝗶𝗯𝗮𝗹𝗶𝘀𝗺 brace bombshell,13.0,4.478117e-01
1362,𝘄𝗶𝘁𝗵 𝘁𝗵𝗲 𝗛𝗮𝗶𝘁𝗶𝗮𝗻 𝗻𝗻𝗶𝗯𝗮𝗹 𝗚𝗮𝗻𝗴,13.0,4.478117e-01
1363,𝘾𝙡𝙖𝙧𝙠𝙨𝙤𝙣 weight loss tips dive,13.0,4.478117e-01
1364,𝙆𝙚𝙡𝙡𝙮 𝘾𝙡𝙖𝙧𝙠𝙨𝙤𝙣 weight loss tips,13.0,4.478117e-01


#### Headline text

In [9]:
results_all_types_headlines = run_stats_test(    
    misinfotext_path="./data/MisInfoText/Analysis_output/MisInfoText_ngrams_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_ngrams_headlines.xlsx"
)
results_all_types_headlines

There are 1193 ngrams that did not change significantly, and 539 ngrams that changed significantly
Total 1732 ngrams


,ngram_text,statistic,pvalue
0,19 deaths,45.0,0.000077
1,19 vaccine,45.0,0.000077
2,19 vaccines,45.0,0.000077
3,aarp,30.0,0.011921
4,abbott,45.0,0.000077
...,...,...,...
1727,zero amish children diagnosed cancer,15.0,0.451417
1728,zika virus,15.0,0.451417
1729,zika virus brazilian,15.0,0.451417
1730,zika virus brazilian microcephaly,15.0,0.451417


In [10]:
results_news_and_blog_headlines = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/news_and_blog/MisInfoText_news_and_blog_ngrams_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/news_and_blog/Fakespeak_news_and_blog_ngrams_headlines.xlsx"
)
results_news_and_blog_headlines

There are 1347 ngrams that did not change significantly, and 439 ngrams that changed significantly
Total 1786 ngrams


,ngram_text,statistic,pvalue
0,19 deaths,45.0,0.000077
1,19 senators voted censor internet,15.0,0.451417
2,19 vaccine,45.0,0.000077
3,19 vaccines,45.0,0.000077
4,abbott,45.0,0.000077
...,...,...,...
1781,zika,15.0,0.451417
1782,zika virus,15.0,0.451417
1783,zika virus brazilian,15.0,0.451417
1784,zika virus brazilian microcephaly,15.0,0.451417


In [11]:
results_press_release_headlines = run_stats_test(
    misinfotext_path="./data/MisInfoText/Analysis_output/press_release/MisInfoText_press_release_ngrams_headlines.xlsx",
    fakespeak_path="./data/Fakespeak-ENG/Analysis_output/press_release/Fakespeak_press_release_ngrams_headlines.xlsx"
)
results_press_release_headlines

There are 1659 ngrams that did not change significantly, and 131 ngrams that changed significantly
Total 1790 ngrams


,ngram_text,statistic,pvalue
0,aarp,30.0,0.011921
1,abandon,15.0,0.451417
2,abortion,30.0,0.011921
3,act,30.0,0.011921
4,administration,45.0,0.000077
...,...,...,...
1785,zemsky,15.0,0.451417
1786,zemsky asking,15.0,0.451417
1787,zemsky asking state,15.0,0.451417
1788,zemsky asking state economic,15.0,0.451417


We'll skip social media headlines because social media mostly doesn't have headlines.